In [1]:
import numpy as np
from numpy.random import random
from copy import  deepcopy

# Load input


In [2]:
from envyron.io.input import Input

In [3]:
natoms = 3
my_input = Input(natoms=natoms, filename='test1.yml')
my_input.ions

IonsModel(atomicspread=[0.5, 0.5, 0.5], corespread=[0.5, 0.5, 0.5], solvationrad=[0.0, 0.0, 0.0])

# Initialize a cell


In [4]:
from envyron.domains.cell import EnvironGrid

In [5]:
L = 20.

at = np.array([
    [L, 0., 0.],
    [0., L, 0.],
    [0., 0., L],
])

nr = np.array([50, 50, 50])

grid = EnvironGrid(at, nr, label='system')
print(grid.origin)
print(grid.lattice)
print(grid.volume)

[0. 0. 0.]
[[20.  0.  0.]
 [ 0. 20.  0.]
 [ 0.  0. 20.]]
8000.0


# Compute minimum image distances


In [6]:
r, r2 = grid.get_min_distance(grid.origin)

# Initialize a density


In [7]:
from envyron.representations import EnvironDensity

In [8]:
density = EnvironDensity(grid, data=random(nr), label='electrons')
print(density.label)
# density.standard_view()

electrons


# Compute multipoles


In [9]:
origin = np.ones(3) * 0.5
density.compute_multipoles(origin)
print(f"charge = {density.charge}")
print(f"dipole = {density.dipole}")
print(f"quadrupole = {density.quadrupole}")

charge = 4008.243862510079
dipole = [379.83198911 433.02336006 374.27406715]
quadrupole = [133646.79668845 133584.62546575 133773.61327042]


# Compute euclidean norm


In [10]:
en = density.euclidean_norm()
# en

# Compute quadratic mean


In [11]:
qm = density.quadratic_mean()
# qm

# Compute scalar product


In [12]:
other_density = EnvironDensity(grid, data=random(nr))
sp = density.scalar_product(other_density)
# sp

# Initialize a gradient


In [13]:
from envyron.representations import EnvironGradient

In [14]:
gradient = EnvironGradient(grid, data=random((3, *nr)), label='electrons')
print(gradient.label)
# gradient.standard_view()

electrons


# Compute modulus


In [15]:
mod = gradient.modulus
print(mod.label)
# mod.standard_view()

electrons_modulus


# Compute scalar products


In [16]:
other_gradient = EnvironGradient(grid, data=random((3, *nr)))
sgp = gradient.scalar_gradient_product(other_gradient)
# sgp.standard_view()

In [17]:
sdp = gradient.scalar_density_product(density)
# sdp

# Initialize a hessian


In [18]:
from envyron.representations import EnvironHessian

In [19]:
hessian = EnvironHessian(grid, data=random((9, *nr)), label='boundary')
print(hessian.label)
# hessian.standard_view()

boundary


# Compute laplacian


In [20]:
lapl = hessian.trace
print(lapl.label)
# hessian.laplacian.standard_view()

boundary laplacian


# Compute scalar product


In [21]:
sgp = hessian.scalar_gradient_product(gradient)
# sgp.standard_view()

# Initialize a gaussian function


In [22]:
from envyron.representations.functions import EnvironGaussian

In [23]:
gaussian = EnvironGaussian(grid, 1, 0, 0, 0.5, 0.5, 0.5, np.array([5, 5, 5]), 'ions')
gaussian.kind

1

# Compute density of function


In [24]:
gd = gaussian.density
print(gd.label)
print(gd.shape)
print(gd.size)
print(gd[gd != 0].size)

ions
(50, 50, 50)
125000
2008


# Compute gradient of function


In [25]:
gg = gaussian.gradient
print(gg.label)
print(gg.shape)
print(gg.size)
print(gg[gg != 0].size)

ions
(3, 50, 50, 50)
375000
6024


# Initialize an erfc function


In [26]:
from envyron.representations.functions import EnvironERFC

In [27]:
erfc = EnvironERFC(grid, 2, 0, 0, 0.5, 0.5, 0.5, np.array([5, 5, 5]), 'soft_spheres')
erfc.kind

2

# Compute density of function


In [28]:
ed = erfc.density
print(ed.label)
print(ed.shape)
print(ed.size)
print(ed[ed != 0].size)

soft_spheres
(50, 50, 50)
125000
118816


# Compute gradient of function


In [29]:
eg = erfc.gradient
print(eg.label)
print(eg.shape)
print(eg.size)
print(eg[eg != 0].size)

soft_spheres
(3, 50, 50, 50)
375000
361920


# Compute laplacian of function


In [30]:
el = erfc.laplacian
print(el.label)
print(el.shape)
print(el.size)
print(el[el != 0].size)

soft_spheres
(50, 50, 50)
125000
120688


# Compute hessian of function


In [31]:
eh = erfc.hessian
print(eh.label)
print(eh.shape)
print(eh.size)
print(eh[eh != 0].size)

soft_spheres
(9, 50, 50, 50)
1125000
1086192


# Compute derivative of function


In [32]:
edv = erfc.derivative
print(edv.label)
print(edv.shape)
print(edv.size)
print(edv[edv != 0].size)

soft_spheres
(50, 50, 50)
125000
120688


# Initialize ions

In [33]:
from envyron.physical import EnvironIons

In [34]:
ions = EnvironIons(
    nions=natoms,
    ntypes=2,
    itypes=[0, 1, 1],
    ion_ids=['O', 'H'],
    zv=[6.0, 1.0],
    atomicspread=my_input.ions.atomicspread,
    corespread=my_input.ions.corespread,
    solvationrad=my_input.ions.solvationrad,
    radius_mode=my_input.solvent.radius_mode,
    is_soft_cavity=False,
    smear=True,
    fill_cores=True,
    grid=grid,
)

In [35]:
coords = np.array([
    [11.79, 12.05, 11.50],
    [13.45, 11.22, 11.50],
    [10.56, 10.66, 11.50],
])

In [36]:
for ion in ions.smeared_ions:
    print(
        ion.label,
        ion.kind,
        ion.dim,
        ion.axis,
        ion.width,
        ion.spread,
        ion.volume,
        ion.pos,
    )

O 1 0 0 0.0 0.5 -6.0 [0. 0. 0.]
H 1 0 0 0.0 0.5 -1.0 [0. 0. 0.]
H 1 0 0 0.0 0.5 -1.0 [0. 0. 0.]


In [37]:
ions.iontypes[1].solvationrad

2.7268748097539834

In [38]:
ions.update(coords)

In [39]:
for ion in ions.smeared_ions:
    print(ion.pos)

[11.79 12.05 11.5 ]
[13.45 11.22 11.5 ]
[10.56 10.66 11.5 ]


In [40]:
print(f"charge     = {ions.charge}")
print(f"center     = {ions.com}")
print(f"quad corr  = {ions.quadrupole_correction}")
print(f"quad pc    = {ions.quadrupole_pc}")
print(f"quad gauss = {ions.quadrupole_gauss}")
print(f"pot shift  = {ions.potential_shift}")

charge     = -8.0
center     = [11.81405814 11.92579284 11.5       ]
quad corr  = -1.0
quad pc    = [-4.25244035e+00 -2.19293955e+00 -2.52435490e-29]
quad gauss = [-5.25244035 -3.19293955 -1.        ]
pot shift  = -0.0015707963267948967


In [41]:
d = ions.density

In [42]:
sd = ions.smeared_ions.density()

In [43]:
sd = ions.smeared_ions[[0, 2]].density()

In [44]:
sd = ions.smeared_ions[0].density

# Initialize system

In [45]:
from envyron.physical import EnvironSystem

In [46]:
system = EnvironSystem(
    ntypes=1,
    dim=0,
    axis=2,
    ions=ions,
)

In [47]:
system.ntypes

1

In [48]:
system.update()

In [49]:
system.com

array([11.79, 12.05, 11.5 ])

# Initialize electrons

In [50]:
from envyron.physical import EnvironElectrons

In [51]:
electrons = EnvironElectrons(grid)

In [52]:
rho = random(nr)

In [53]:
electrons.update(rho / 1000)
print(f"charge = {electrons.charge}")
print(f"count  = {electrons.count}")

charge = 4.000118142167143
count  = 4


# Initialize a core container

In [54]:
from envyron.cores import CoreContainer

In [55]:
cores = CoreContainer(label='outer', has_internal_correction=False)

# Initialize an electronic boundary

In [56]:
from envyron.boundaries import ElectronicBoundary

In [57]:
eboundary = ElectronicBoundary(
    rhomin=my_input.solvent.rhomin,
    rhomax=my_input.solvent.rhomax,
    electrons=electrons,
    mode='electronic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [58]:
# electrons.updating = True
eboundary.update()
# electrons.updating = False

In [59]:
print(f"volume  = {eboundary.volume}")
print(f"surface = {eboundary.surface}")

volume  = 0.0
surface = 0.0


# Initialize an ionic boundary

In [60]:
from envyron.boundaries import IonicBoundary

In [61]:
iboundary = IonicBoundary(
    alpha=1.12,
    softness=0.5,
    ions=ions,
    mode='ionic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='lowmem',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [62]:
ions.updating = True
iboundary.update()
ions.updating = False

In [63]:
print(f"volume  = {iboundary.volume}")
print(f"surface = {iboundary.surface}")

volume  = 281.8079914165231
surface = 209.31941232992193


# Initialize an system boundary

In [64]:
from envyron.boundaries import SystemBoundary

In [65]:
sboundary = SystemBoundary(
    distance=my_input.solvent.distance,
    spread=my_input.solvent.spread,
    system=system,
    mode='system',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [66]:
system.updating = True
sboundary.update()
system.updating = False

In [67]:
print(f"volume  = {sboundary.volume}")
print(f"surface = {sboundary.surface}")

volume  = 5.759635791400076
surface = 14.136621232521021
